---
title: "PyTorch Exercises"
jupyter: python3
---


## Imports 

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from ml_utils import plot_decision_boundary

## PyTorch Exercises 00 pytorch fundamentals
https://www.learnpytorch.io/00_pytorch_fundamentals/#exercises

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(1234)
x = torch.randn(7 ,7)
y = torch.randn(1 ,7)
z = torch.matmul(x, y.T)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
gpu_1 = torch.randn(2 ,3).to(device)
torch.manual_seed(42)
gpu_2 = torch.randn(2 ,3).to(device)

gpu_3 = torch.matmul(gpu_1, gpu_2.T)

print(gpu_3.max())
print(gpu_3.argmax())
print(gpu_3.reshape(4)[3])

print(gpu_3.argmin())

torch.manual_seed(7)
original_tensor = torch.randn(1, 1, 1, 10)
squeezed_tensor = original_tensor.squeeze()
print(original_tensor.shape, original_tensor)
print(squeezed_tensor.shape, squeezed_tensor)

## PyTorch Exercises 01 pytorch Workflow fundamentals

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = torch.arange(0, 1, 0.0025, dtype=torch.float32).unsqueeze(dim=1).to(device)
print(x.shape)
target_bias = 0.9
target_weight = 0.3
y = target_bias + target_weight * x

training_split = int(0.8 * len(x))

x_train, y_train = x[:training_split], y[:training_split]
x_val, y_val = x[training_split:], y[training_split:]

plt.scatter(x_train.cpu(), y_train.cpu(), label="train", color="blue")
plt.scatter(x_val.cpu(), y_val.cpu(), label="validation", color="orange")
plt.legend()
plt.xlabel("x")
plt.ylabel("y")
plt.show()

class LinearRegression(nn.Module):
  def __init__(self):
    super().__init__()
    self.weight = nn.Parameter(torch.randn(1, device=device))
    self.bias = nn.Parameter(torch.randn(1, device=device))
    
  def forward(self, x):
    return self.weight * x + self.bias

torch.manual_seed(2)
model_0 = LinearRegression()

loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(model_0.parameters(), lr=0.02)

nun_epochs = 300 

epochs = []
train_losses = []
val_losses = []

for epoch in range(nun_epochs):
  model_0.train()
  
  y_logits = model_0(x_train)
  
  loss = loss_fn(y_logits, y_train)

  optimizer.zero_grad()

  loss.backward()

  optimizer.step()
  
  model_0.eval()
  
  with torch.inference_mode():
    y_val_logits = model_0(x_val)
    val_loss = loss_fn(y_val_logits, y_val)
  
  epochs.append(epoch)
  train_losses.append(loss.item())
  val_losses.append(val_loss.item())
  
  if epoch % 20 == 0:
    print(f"epoch {epoch+1}: train loss {loss}, val loss {val_loss}")

plt.plot(epochs, train_losses, label="train")
plt.plot(epochs, val_losses, label="val")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.legend()
plt.show()

with torch.inference_mode():
  y_logits = model_0(x_val)

plt.scatter(x_train.detach().cpu(), y_train.detach().cpu(), label="train", color="green")
plt.scatter(x_val.cpu(), y_val.cpu(), label="validation", color="orange")
plt.scatter(x_val.cpu(), y_logits.cpu(), label="predictions", color="blue")
plt.legend()
plt.xlabel("x")
plt.ylabel("y")
plt.show()

torch.save(model_0.state_dict(), "model_0.pth")

model_0_loaded = LinearRegression()
model_0_loaded.load_state_dict(torch.load("model_0.pth"))
with torch.inference_mode():
  y_logits = model_0_loaded(x_val)

plt.scatter(x_train.detach().cpu(), y_train.detach().cpu(), label="train", color="green")
plt.scatter(x_val.cpu(), y_val.cpu(), label="validation", color="orange")
plt.scatter(x_val.cpu(), y_logits.cpu(), label="predictions", color="blue")
plt.legend()
plt.xlabel("x")
plt.ylabel("y")
plt.show()

## PyTorch Exercises 02 pytorch classification

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

NUM_SAMPLES = 1000
RANDOM_SEED = 42
X, y = make_moons(NUM_SAMPLES, noise=0.2, random_state=RANDOM_SEED)

X = torch.from_numpy(X).float().to(device)
y = torch.from_numpy(y).float().to(device)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

plt.subplot(1, 2, 1)
plt.scatter(X_train.cpu()[:, 0], X_train.cpu()[:, 1], c=y_train.cpu(), cmap=plt.cm.RdYlBu)
plt.subplot(1, 2, 2)
plt.scatter(X_test.cpu()[:, 0], X_test.cpu()[:, 1], c=y_test.cpu(), cmap=plt.cm.RdYlBu)
plt.show()

class LogisticRegression(nn.Module):
  def __init__(self, in_features, out_features):
    super().__init__()
    self.sequential = nn.Sequential(
      nn.Linear(in_features, 64),
      nn.ReLU(),
      nn.Linear(64, 64),
      nn.ReLU(),
      nn.Linear(64, out_features),
    )
  def forward(self, x):
    return self.sequential(x).squeeze(-1)

torch.manual_seed(42)
torch.cuda.manual_seed(42)
model1 = LogisticRegression(2, 1).to(device)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model1.parameters(), lr=0.01)

NUM_EPOCHS = 10000

epochs = []
train_losses = []
val_losses = []
train_accs = []
val_accs = []
 
for epoch in range(NUM_EPOCHS):
  model1.train()
  
  y_logits = model1(X_train.squeeze())
  y_preds = torch.round(torch.sigmoid(y_logits))
  
  train_acc = accuracy_score(y_train.detach().cpu(), y_preds.detach().cpu())
  train_accs.append(train_acc)
  
  loss = loss_fn(y_logits, y_train)
  
  optimizer.zero_grad()
  
  loss.backward()
  
  optimizer.step()
  
  model1.eval()
  
  with torch.inference_mode():
    y_val_logits = model1(X_test)
    y_preds = torch.round(torch.sigmoid(y_val_logits))
    val_acc = accuracy_score(y_test.detach().cpu(), y_preds.detach().cpu())
    val_accs.append(val_acc)
    val_loss = loss_fn(y_val_logits, y_test)
  
  epochs.append(epoch)
  train_losses.append(loss.item())
  val_losses.append(val_loss.item())
  
  if epoch % 10 == 0:
    print(f"epoch {epoch+1}: train loss {loss} | train acc {train_acc} |  val loss {val_loss} | val acc {val_acc}")

plot_decision_boundary(model1, X_train, y_train)
plt.show()

def tanh(x):
  return (torch.exp(x) - torch.exp(-x)) / (torch.exp(x) + torch.exp(-x))

# Multiclass classification for spiral dataset from CS231n
POINTS_PER_CLASS = 100
DIMENSIONALITY = 2
NUM_CLASSES = 3
X = torch.zeros(POINTS_PER_CLASS * NUM_CLASSES, DIMENSIONALITY)
y = torch.zeros(POINTS_PER_CLASS * NUM_CLASSES, dtype=torch.long)

for class_number in range(NUM_CLASSES):
  ix = range(POINTS_PER_CLASS * class_number, POINTS_PER_CLASS * (class_number + 1))
  r = torch.linspace(0.0, 1, POINTS_PER_CLASS)
  t = torch.linspace(class_number * 4, (class_number + 1) * 4, POINTS_PER_CLASS) + torch.randn(POINTS_PER_CLASS) * 0.2
  X[ix] = torch.stack([r * torch.cos(t), r * torch.sin(t)], dim=1)
  y[ix] = class_number

plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.Spectral)
plt.show()

torch.manual_seed(42)
torch.cuda.manual_seed(42)
model2 = LogisticRegression(DIMENSIONALITY, NUM_CLASSES).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=0.01)

X = X.to(device)
y = y.to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

NUM_EPOCHS = 1000

epochs = []
train_losses = []
val_losses = []
train_accs = []
val_accs = []

for epoch in range(NUM_EPOCHS):
  model2.train()
  
  y_logits = model2(X_train)
  y_preds = torch.argmax(y_logits, dim=1)
  
  train_acc = accuracy_score(y_train.detach().cpu(), y_preds.detach().cpu())
  train_accs.append(train_acc)
  
  loss = loss_fn(y_logits, y_train)
  
  optimizer.zero_grad()
  
  loss.backward()
  
  optimizer.step()
  
  model2.eval()
  
  with torch.inference_mode():
    y_val_logits = model2(X_test)
    y_val_preds = torch.argmax(y_val_logits, dim=1)
    val_acc = accuracy_score(y_test.detach().cpu(), y_val_preds.detach().cpu())
    val_accs.append(val_acc)
    val_loss = loss_fn(y_val_logits, y_test)
  
  epochs.append(epoch)
  train_losses.append(loss.item())
  val_losses.append(val_loss.item())
  
  if epoch % 10 == 0:
    print(f"epoch {epoch+1}: train loss {loss} | train acc {train_acc} |  val loss {val_loss} | val acc {val_acc}")

plot_decision_boundary(model2, X_train, y_train)
plt.show()